In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [94]:
path = '/content/drive/My Drive/AI Stuff/projects/MIT CBMM/NSD/'
# %cd
!ls

In [187]:
import numpy as np
import pandas as pd
from pathlib import Path
import h5py
from scipy.stats import pearsonr
import pickle
from tqdm.notebook import tqdm

# Steps:

1. get a list of images
2. how to extract fmri from the corresponding hdf5 file
3.  If hdf5 extraction is fast, do extraction of fmris on the fly (optimization of hdf5 extraction)
4. (Later) apply ROI
5. write the RDM code
6. slurmify the code for all subjects

## 1. get a list of images to be worked on

In [110]:
#download the shared_1000_info.csv file from here: https://github.com/PalaashAgrawal/NSD_exploration/blob/main/shared_1000_info.csv
info_file_path = Path(path)/'shared1000 extraction/shared_1000_info.csv'
session_file_path = Path(path)/'basic fMRI visualization' #where do you want to save the session files (hdf5 files of the fmris)

In [111]:
info_file = pd.read_csv(info_file_path)
assert len(info_file)==1000
info_file.head()

,Unnamed: 0,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1_rep0,...,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2,coco_label,category,supercategory
0,2950,262145,train2017,"(0, 0, 0.16640625, 0.16640625)",0.09375,2950,False,True,True,2616,...,27566,2616,9716,27566,2616,9716,27566,1,person,person
1,2990,262239,train2017,"(0, 0, 0.1671875, 0.1671875)",0.10000,2990,False,True,True,18458,...,27711,18458,18697,27711,18458,18697,27711,1,person,person
2,3049,262414,train2017,"(0, 0, 0.125, 0.125)",0.00000,3049,False,True,True,6299,...,6697,6299,6448,6697,6299,6448,6697,1,person,person
3,3077,524646,train2017,"(0, 0, 0.1671875, 0.1671875)",0.00000,3077,False,True,True,4289,...,4537,4289,4515,4537,4289,4515,4537,1,person,person
4,3146,262690,train2017,"(0, 0, 0.16640625, 0.16640625)",0.00000,3146,False,True,True,8087,...,26807,8087,8443,26807,8087,8443,26807,85,clock,indoor


In [112]:
max(info_file[["subject1_rep0", "subject1_rep1", "subject1_rep2"]].max())


29966

we're interested in creating a mapping stim_0001 to the indices of the stimulus during the experiments. In this case: stim_0001 corresponds to the first stimulus from the shared 1000 set that was shown to the subject in the very first repetition (rep0). The mapping is of the form:

`stim = {1: (cocoID, [rep0id,rep1id,rep2id], 
         2: ()....)` 

Since the last 3 sessions are held out (= the last 3x750 = 2250 fmris) (ie, we only have a total of 27750 fmris), we cant include scans beyond that index for now. Also we'll have to discard the entire scan if even one repetition is not available, because otherwise different stimuli fmris will have different sizes, and correlation cannot be calculated in that case. 

So, for now, we will only build RDMs out of the first repetition, and not others, and hence we only consider those stimuli whose first repetition (the earliest iteration) is provided in the dataset of 27750 fmris.). 

This also means that different patients will have different sets of images for which RDMs are calculated. Also, that would anyways be true because not all the subjects saw all the pictures. Some didnt even see the shared 1000 images even once (see [here](https://cvnlab.slite.com/p/channel/CPyFRAyDYpxdkPK6YbB5R1/notes/h_T_2Djeid)). Only 907 of the 1000 images were seen by all the patients even once. But lets go with that for now.

TALK TO KAMILA LATER ABOUT WHAT TO DO. 

In [113]:
subject=1 #which human subject are we working on

In [114]:
fmri_order_list=[]
for i,vals in (info_file.iterrows()): 
    # fmri_order_list.append([vals[f'subject{subject}_rep{rep}'] for rep in range(3)]) #get list of rep0,rep1 and rep2 values for each row
    order = vals[f'subject{subject}_rep0']
    if order<=27750: fmri_order_list.append(order)
fmri_order_list = sorted(fmri_order_list) #, key = lambda x: x[0]) #use key when you're appending a list of 3 items in the fmri_order_list
stim_subj1 = {i: order for i,order in enumerate(fmri_order_list)}

In [115]:
stim_subj1

{0: 1,
 1: 29,
 2: 36,
 3: 45,
 4: 56,
 5: 57,
 6: 121,
 7: 127,
 8: 133,
 9: 153,
 10: 167,
 11: 174,
 12: 177,
 13: 202,
 14: 232,
 15: 233,
 16: 246,
 17: 247,
 18: 250,
 19: 251,
 20: 256,
 21: 268,
 22: 282,
 23: 306,
 24: 346,
 25: 357,
 26: 360,
 27: 374,
 28: 403,
 29: 434,
 30: 441,
 31: 449,
 32: 469,
 33: 498,
 34: 505,
 35: 520,
 36: 524,
 37: 528,
 38: 575,
 39: 601,
 40: 629,
 41: 634,
 42: 667,
 43: 686,
 44: 702,
 45: 732,
 46: 748,
 47: 753,
 48: 761,
 49: 763,
 50: 781,
 51: 782,
 52: 803,
 53: 809,
 54: 828,
 55: 839,
 56: 870,
 57: 881,
 58: 898,
 59: 901,
 60: 908,
 61: 914,
 62: 926,
 63: 944,
 64: 951,
 65: 994,
 66: 997,
 67: 999,
 68: 1011,
 69: 1018,
 70: 1029,
 71: 1092,
 72: 1111,
 73: 1120,
 74: 1125,
 75: 1151,
 76: 1159,
 77: 1177,
 78: 1178,
 79: 1186,
 80: 1197,
 81: 1209,
 82: 1225,
 83: 1250,
 84: 1259,
 85: 1262,
 86: 1288,
 87: 1305,
 88: 1320,
 89: 1332,
 90: 1339,
 91: 1347,
 92: 1356,
 93: 1373,
 94: 1391,
 95: 1394,
 96: 1405,
 97: 1425,
 98: 14

## step2: getting fmri from stimuli order

Run the below functions once to download the files needed to run this notebook. For this notebook, we only download two files

In [173]:
# !pip install wget >./tmp
# import wget
# file1 = "https://natural-scenes-dataset.s3.amazonaws.com/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session01.hdf5"
# file2 = "https://natural-scenes-dataset.s3.amazonaws.com/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session02.hdf5"
# wget.download(file1,str(session_file_path))
# wget.download(file2,str(session_file_path))
!ls "{session_file_path}"

 betas_session01.hdf5	    'fmri visualization.ipynb'
 betas_session01.nii.gz      lh.prf-visualrois.nii.gz
'betas_session02 (1).hdf5'   rough.ipynb
 betas_session02.hdf5	     tmp
 floc_adulttval.nii.gz


In [60]:
def get_fmri_from_stimulus_order(stimulus_order, subj:int):
    f'''
    get fmri from the hdf5 files given the stimulus_order d which subject "subj" is being considered
    '''
    stimulus_order = list(stimulus_order)
    stimulus_order=[order-1 for order in stimulus_order] 
    #by default stimulus_order is 1-indexed. 0s are used to represent that the image was not shown at all. 
    #But no values are 0 in the shared1000, because we have only included images which were shown to each participant.
    #So for easier computation, convert everything to 0-indexed
    session = [order//750 + 1 for order in stimulus_order] #each session is a 750 size session. Also they are 1-indexed, hence the plus 1. 
    index = [order%750 for order in stimulus_order]
    # session_file = session_file_path/f'subj{subj:02d}/betas_session{session:02d}.hdf5'
    session_file = 'betas_session01.hdf5' #for now, we're testing on only one hdf5 file. Uncomment this in openmind
    with h5py.File(session_file,"r") as f: 
        beta = f['betas'][()] #its usually faster to access the whole set than to slice it directly. 
        return beta[index]

In [61]:
%time f = get_fmri_from_stimulus_order([56,43,132,43,12,5],1);

CPU times: user 6.95 s, sys: 1.34 s, total: 8.29 s
Wall time: 8.91 s


### Optimization

If I were to run this code for 1000 images, I would be looking at something close to 7 seconds x 1000 = 700 minutes. Which is way too long just to extract the fmris for a subject. So we need to optimize it. Thinking of chunking the dataset, by grouping all images that can be extracted from a single hdf5 file together. So we're looking at a max 7 seconds x 40 scan sessions = 280 seconds = 4.5 minutes. Quite better. Thats the max we can go!

Algorithm: 

1. take the session dictionary
2. run a chunkify function that acts as a generator that returns list of stimulus_order that belong to the same hdf5 file
3. This function will return the session number and the index list
4. collect all the fmris together and return a list of all fmris for the entire session dictionary. 

In [157]:
def get_chunkified_stimulus_order(stimulus_order:dict):
    session = 0
    order_list = []
    for stim_idx,order in stimulus_order.items(): 
        if order//750 == session and stim_idx!=len(stimulus_order)-1:
            order_list.append((order-1)%750) #(order-1)%750 to get the zero index slice value for the corresponding session
        else: #either session has changed or we have reached the end of the stimulus_order dictionary
            yield session+1, order_list #note that session files are 1-indexed
            session+=1
            order_list=[(order-1)%750]

In [158]:
# testing if this works
for session, order_list in get_chunkified_stimulus_order(stim_subj1): 
    print(session,order_list)
    break

1 [0, 28, 35, 44, 55, 56, 120, 126, 132, 152, 166, 173, 176, 201, 231, 232, 245, 246, 249, 250, 255, 267, 281, 305, 345, 356, 359, 373, 402, 433, 440, 448, 468, 497, 504, 519, 523, 527, 574, 600, 628, 633, 666, 685, 701, 731, 747]


Rebuilding the get_fmri function. This time it returns a list

In [159]:
def get_fmri_list(stimulus_order:dict, subj:int): #we arent using subj in our function, but we'll use it in openmind, where each fmri betas are stored under the corresponding subj folder
    f'''
    get list of fmri's from the hdf5 files given the stimulus_order dictionary which corresponds to the subject "subj" is being considered
    '''
    fmri_list = []
    for session, indices in get_chunkified_stimulus_order(stimulus_order):
        session_file = session_file_path/f'betas_session{session:02d}.hdf5'
        with h5py.File(session_file,"r") as f: 
            beta = f['betas'][()]
            fmri_list.extend(list(beta[indices]))
    return fmri_list

In [167]:
stim_subj1_subset = {i:stim_subj1[i] for i in stim_subj1 if stim_subj1[i]< 750*2} #we've only downloaded 2 files as of now

In [166]:
%time fmris = get_fmri_list(stim_subj1_subset,1)

[0, 28, 35, 44, 55, 56, 120, 126, 132, 152, 166, 173, 176, 201, 231, 232, 245, 246, 249, 250, 255, 267, 281, 305, 345, 356, 359, 373, 402, 433, 440, 448, 468, 497, 504, 519, 523, 527, 574, 600, 628, 633, 666, 685, 701, 731, 747]
[2, 10, 12, 30, 31, 52, 58, 77, 88, 119, 130, 147, 150, 157, 163, 175, 193, 200, 243, 246, 248, 260, 267, 278, 341, 360, 369, 374, 400, 408, 426, 427, 435, 446, 458, 474, 499, 508, 511, 537, 554, 569, 581, 588, 596, 605, 622, 640, 643, 654, 674, 689, 694, 711]
CPU times: user 18.9 s, sys: 4.07 s, total: 23 s
Wall time: 28.9 s


## Step3: ROI application

## Step4: RDM Extraction + saving

In [188]:
def construct_RDM(activations):
    num_images = len(activations)
    RDM = np.zeros((num_images, num_images))

    for x in tqdm(range(num_images)):
        for y in range(num_images):
            if x<=y: #because they're symmetric
                # get the pearson correlation
                correl = 1 - (pearsonr(activations[x].flatten(), activations[y].flatten()))[0]
                RDM[x][y] = correl
                RDM[y][x] = correl
    return RDM

In [189]:
%time rdm = construct_RDM(fmris)
rdm

  0%|          | 0/101 [00:00<?, ?it/s]

CPU times: user 1min 1s, sys: 48.2 s, total: 1min 49s
Wall time: 59.6 s


array([[0.00000000e+00, 9.07254377e-01, 8.76009798e-01, ...,
        9.79969888e-01, 9.85968884e-01, 9.48322201e-01],
       [9.07254377e-01, 1.99840144e-14, 9.08732129e-01, ...,
        9.79967147e-01, 9.75911394e-01, 9.55495662e-01],
       [8.76009798e-01, 9.08732129e-01, 6.32827124e-14, ...,
        9.75340205e-01, 9.76877210e-01, 9.45405661e-01],
       ...,
       [9.79969888e-01, 9.79967147e-01, 9.75340205e-01, ...,
        4.19664303e-14, 9.25561013e-01, 9.76233210e-01],
       [9.85968884e-01, 9.75911394e-01, 9.76877210e-01, ...,
        9.25561013e-01, 6.68354261e-14, 9.54735286e-01],
       [9.48322201e-01, 9.55495662e-01, 9.45405661e-01, ...,
        9.76233210e-01, 9.54735286e-01, 0.00000000e+00]])

In [190]:
rdm.shape

(101, 101)

In [ ]:
#done!